### 볼린져밴드 + 절대모멘텀 
1. 첫번째 함수 
    - 이동평균선 , 상단밴드, 하단밴드 생성
    - 년-월 컬럼을 생성
    - 투자 기간을 이용하여 필터링
2. 두번째 함수 
    - 절대모멘텀 월말데이터를 추출하여 새로운 데이터프레임을 생성
    - 전월의 주가, 전년도 주가 컬럼을 생성
3. 세번째 함수 
    - 볼린져밴드 투자전략으로 구매 타이밍 생성 (boll_trade)
    - 절대 모멘텀 전략으로 구매타이밍 생성 (momentum_trade)
4. 네번째 함수 
    - 세번째 함수에서 생성된 구매 타이밍을 기준으로 해서 실제 구매 타이밍(trade)
5. 다섯번째 함수
    - trade를 기준으로 하여 수익율 계산

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
def create_band_ym(
        _df, 
        _col = 'Adj Close', 
        _roll = 20):
    # 컬럼의 Date가 존재한다면 
    if 'Date' in _df.columns:
        _df.set_index('Date', inplace=True)
    # index를 시계열데이터로 변경 
    _df.index = pd.to_datetime(_df.index)

    # 특정 컬럼을 제외하고 모두 제거 
    _df = _df[[_col]].copy()
    # 결측치, 무한대 데이터를 제외
    flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    _df = _df.loc[~flag]

    # 이동평균선을 생성 
    _df['center'] = _df[_col].rolling(_roll).mean()
    # 상단 밴드, 하단 밴드 생성
    _df['ub'] = _df['center'] + ( 2 * _df[_col].rolling(_roll).std() )
    _df['lb'] = _df['center'] - ( 2 * _df[_col].rolling(_roll).std() )

    # 년-월 (STD-YM) 컬럼을 생성 
    _df['STD-YM'] = _df.index.map(
        lambda x : x.strftime('%Y-%m')
    )

    return _df

In [ ]:
df = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
price_df = create_band_ym(df)

In [ ]:
price_df.head()

In [ ]:
# 두번째 함수
def create_month_last(_df, 
                      _momentum = 12, 
                      _start = "2010-01-01", 
                      _end = "2023-12-31"):
    # 새로운 데이터프레임을 생성
    result = pd.DataFrame()
    ym_list = _df['STD-YM'].unique()

    # 월말 데이터를 추출하여 result에 대입 
    for i in ym_list:
        flag = _df['STD-YM'] == i
        # data = _df.loc[flag].tail(1)
        data = _df.loc[flag].iloc[[-1]]
        result = pd.concat([result, data], axis=0)
        # result.append(data)

    # 기준이 되는 컬럼의 이름을 추출 
    col = _df.columns[0]

    # 전월의 주가 컬럼 생성
    result['BF1'] = result.shift(1)[col].fillna(0)
    # _momentum만큼 이동한 월의 주가 컬럼 생성
    result['BF2'] = result.shift(_momentum)[col].fillna(0)

    # 시간시간과 종료시간으로 필터링 
    _df = _df.loc[_start:_end].copy()
    result = result.loc[_start:_end]
    
    return _df, result
    

In [ ]:
price_df, month_last_df = create_month_last(price_df)

In [ ]:
price_df.head(5)

In [ ]:
month_last_df.head(5)

In [ ]:
# 세번째 함수 
def create_multi_trade(_df, _df2, _score = 1):
    # 기준이되는 주가 컬럼 
    col = _df.columns[0]
    _df = _df.copy()
    # 볼린져밴드를 이용한 구매 타이밍
    _df['boll_trade'] = ""

    # 구매 타이밍을 추가 
    for i in _df.index:
        # 상단밴드보다 주가가 높은 경우
        if _df.loc[i, col] > _df.loc[i, 'ub']:
            _df.loc[i, 'boll_trade'] = ""
        # 하단밴드보다 주가가 낮은 경우 
        elif _df.loc[i, col] < _df.loc[i, 'lb']:
            _df.loc[i, 'boll_trade'] = 'buy'
        # 주가가 밴드 사이에 있다면
        else:
            if _df.shift().loc[i, 'boll_trade'] == 'buy':
                _df.loc[i, 'boll_trade'] = 'buy'
            else:
                _df.loc[i, 'boll_trade'] = ''
    
    # 모멘텀 구매 타이밍 추가 
    _df['momentum_trade'] = ""

    # 모멘텀 인덱스를 생성 
    for i in _df2.index:
        signal = ""

        # 모멘텀 인덱스 계산 ( 전월의 주가 / 전년도 주가 - 1 )
        momentum_index = _df2.loc[i, 'BF1'] / _df2.loc[i, 'BF2'] - _score

        # 모멘텀 인덱스가 0보다 크고 무한대 아닌 조건식 
        flag = (momentum_index > 0) & (momentum_index != np.inf)

        if flag:
            signal = "buy"
        
        _df.loc[i: , 'momentum_trade'] = signal

    return _df


In [ ]:
book = create_multi_trade(price_df, month_last_df)

In [ ]:
book.tail(10)

In [ ]:
# book에서 trade 컬럼을 추가하고
# boll_trade가 buy 이고 momentum_trade가 'buy'인 경우 trade에 'buy' 대입

def create_trade(_df):
    _df = _df.copy()
    _df['trade_and'] = ""
    _df['trade_or'] = ""

    for i in _df.index:
        flag_and = (_df.loc[i, 'boll_trade'] == 'buy') & \
            (_df.loc[i, 'momentum_trade'] == 'buy') 
        flag_or = (_df.loc[i, 'boll_trade'] == 'buy') | \
            (_df.loc[i, 'momentum_trade'] == 'buy')
        if flag_and:
            _df.loc[i, 'trade_and'] = 'buy'
        if flag_or:
            _df.loc[i, 'trade_or'] = 'buy'
    return _df

In [ ]:
book2 = create_trade(book)

In [ ]:
book2['trade_and'].value_counts()

In [ ]:
book2['trade_or'].value_counts()

In [83]:
# 수익율 계산 함수 
def create_rtn(_df):
    # 기준이 되는 컬럼명 
    col = _df.columns[0]

    _df['rtn_trade_and'] = 1
    _df['rtn_trade_or'] = 1
    buy = dict()
    sell = dict()
    _list =[]

    # 수익율 대입
    for i in _df.index:
        for j in ['trade_and', 'trade_or']:
            # 매수
            if (_df.loc[i, j] == "buy") & (_df.shift().loc[i, j] == ""):
                buy[j] = _df.loc[i, col]
            # 매도
            elif (_df.loc[i, j] == "") & (_df.shift().loc[i, j] == 'buy'):
                sell[j] = _df.loc[i, col]
                rtn = sell[j] / buy[j] 
                _df.loc[i, "rtn_"+j] = rtn
                _list.append([buy, sell])
    # 누적 수익율 계산
    _df['acc_rtn_and'] = _df['rtn_trade_and'].cumprod()
    _df['acc_rtn_or'] = _df['rtn_trade_or'].cumprod()

    print(f"AND의 누적 수익율 : {_df['acc_rtn_and'].iloc[-1]}")
    print(f"OR의 누적 수익율 : {_df['acc_rtn_or'].iloc[-1]}")
    return _df

In [ ]:
book2, trade_list =  create_rtn(book2)

In [ ]:
trade_list

In [ ]:
# yfinance라이브러리를 이용해서 주식 데이터 로드 
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
AAPL = yf.download('005930.KS', start = "2010-01-01")

In [ ]:
AAPL.head()

In [ ]:
AAPL.tail()

In [ ]:
AAPL = create_band_ym(AAPL)

In [ ]:
AAPL, AAPL_last = create_month_last(AAPL, _start="2021-01-01")

In [ ]:
book = create_multi_trade(AAPL, AAPL_last)
book.head(2)

In [ ]:
book = create_trade(book)
book.head(2)

In [84]:
book = create_rtn(book)

C:\Users\moons\AppData\Local\Temp\ipykernel_13716\1853620699.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0220047205045621' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  _df.loc[i, "rtn_"+j] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_13716\1853620699.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9723181430106024' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  _df.loc[i, "rtn_"+j] = rtn


AND의 누적 수익율 : 1.0067262354696256
OR의 누적 수익율 : 0.8781732548467625
